In [1]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import nltk
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix, roc_curve, auc
from nltk.stem.porter import PorterStemmer

Connect to the database

In [2]:
# load sqlite database
con = sqlite3.connect(r'C:\Users\ucanr\aaic\database.sqlite')

In [19]:
df = pd.read_sql_query("select * from reviews where score <> 3;", con)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525814 entries, 0 to 525813
Data columns (total 10 columns):
Id                        525814 non-null int64
ProductId                 525814 non-null object
UserId                    525814 non-null object
ProfileName               525814 non-null object
HelpfulnessNumerator      525814 non-null int64
HelpfulnessDenominator    525814 non-null int64
Score                     525814 non-null int64
Time                      525814 non-null int64
Summary                   525814 non-null object
Text                      525814 non-null object
dtypes: int64(5), object(5)
memory usage: 40.1+ MB


* Mark reviews with rating > 3 as positive
* Mark reviews with rating < 3 as negative

In [21]:
m1 =  df['Score'] > 3 
m2 =  df['Score'] < 3 

df['Score'] = np.select([m1,m2], ['positive','negative'])

In [24]:
df.Score.value_counts()

positive    443777
negative     82037
Name: Score, dtype: int64

Using Categories instead of Object reduces the memory requirement

In [31]:
df['Score']=df['Score'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525814 entries, 0 to 525813
Data columns (total 10 columns):
Id                        525814 non-null int64
ProductId                 525814 non-null object
UserId                    525814 non-null object
ProfileName               525814 non-null object
HelpfulnessNumerator      525814 non-null int64
HelpfulnessDenominator    525814 non-null int64
Score                     525814 non-null category
Time                      525814 non-null int64
Summary                   525814 non-null object
Text                      525814 non-null object
dtypes: category(1), int64(4), object(5)
memory usage: 36.6+ MB


In [25]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Are there any duplicate rows in the dataset?

In [35]:
df.duplicated('Text').value_counts()

False    363836
True     161978
dtype: int64

In [45]:
df.drop(df[df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']].index.tolist(), axis=0, inplace=True)

In [56]:
# Verify whether the rows have been dropped
df [df['HelpfulnessNumerator'] > df['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,is_duplicate


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525812 entries, 0 to 525813
Data columns (total 11 columns):
Id                        525812 non-null int64
ProductId                 525812 non-null object
UserId                    525812 non-null object
ProfileName               525812 non-null object
HelpfulnessNumerator      525812 non-null int64
HelpfulnessDenominator    525812 non-null int64
Score                     525812 non-null category
Time                      525812 non-null int64
Summary                   525812 non-null object
Text                      525812 non-null object
is_duplicate              525812 non-null bool
dtypes: bool(1), category(1), int64(4), object(5)
memory usage: 61.1+ MB


In [61]:
%timeit df.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep = 'first', inplace=True)

810 ms ± 47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364171 entries, 0 to 525813
Data columns (total 11 columns):
Id                        364171 non-null int64
ProductId                 364171 non-null object
UserId                    364171 non-null object
ProfileName               364171 non-null object
HelpfulnessNumerator      364171 non-null int64
HelpfulnessDenominator    364171 non-null int64
Score                     364171 non-null category
Time                      364171 non-null int64
Summary                   364171 non-null object
Text                      364171 non-null object
is_duplicate              364171 non-null bool
dtypes: bool(1), category(1), int64(4), object(5)
memory usage: 28.5+ MB


Sort data by Time

In [63]:
df.sort_values('Time', ascending=True,inplace=True)

In [65]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,is_duplicate
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,False
138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,positive,940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,False
417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,positive,944092800,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,False
212472,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,positive,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",False
417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,positive,946857600,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,False
212533,230348,B00004RYGX,A1048CYU0OV4O8,Judy L. Eans,2,2,positive,947376000,GREAT,THIS IS ONE MOVIE THAT SHOULD BE IN YOUR MOVIE...,False
212458,230269,B00004RYGX,A1B2IZU1JLZA6,Wes,19,23,negative,948240000,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very...",False
70688,76882,B00002N8SM,A32DW342WBJ6BX,Buttersugar,0,0,positive,948672000,A sure death for flies,I bought a few of these after my apartment was...,False
212558,230376,B00004RYGX,ACJR7EQF9S6FP,Jeremy Robertson,2,3,positive,951523200,Bettlejuice...Bettlejuice...BETTLEJUICE!,What happens when you say his name three times...,False
212511,230326,B00004RYGX,A2DEE7F9XKP3ZR,jerome,0,3,positive,959990400,Research - Beatlejuice video - French version,I'm getting crazy.I'm looking for Beatlejuice ...,False


### Text Preprocessing